In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from TME01.Parser import Parser
from TME03.QueryParser import QueryParser
from TME01.IndexerSimple import IndexerSimple
from TME02.Weighter import Weighter1,Weighter2,Weighter3,Weighter4,Weighter5
from TME02.Vectoriel import Vectoriel
from TME02.ModeleLangue import ModeleLangue
from TME02.Okapi import Okapi
from TME03.EvalMesure import Precision, Rappel,Fmesure, AvgP,ReciprocalRank,NDCG
from TME03.EvalIRModel import EvalIRModel
import pickle as pkl
import pandas as pd
import numpy as np
import random

#### Parsing des collections de documents

In [3]:
### Parsing des collections et des requetes correspondantes
parsedCollection_cacm =Parser('data/cacm/cacm.txt').dict_doc
parsedCollection_cisi =Parser('data/cisi/cisi.txt').dict_doc

#### Construction des index sur les deux collections

In [15]:
ind_cacm=IndexerSimple(parsedCollection_cacm)
ind_cisi=IndexerSimple(parsedCollection_cisi)

#### Parsing des collections de requetes

In [16]:
parsedQuery_cacm =QueryParser('data/cacm/cacm.qry','data/cacm/cacm.rel').dict_q
parsedQuery_cisi =QueryParser('data/cisi/cisi.qry','data/cacm/cacm.rel').dict_q

#### Ecriture des index & des index inverses dans des fichiers .txt

In [ ]:
### Ecriture des index dans des fichiers .txt
fichier_index=open('index_cacm.txt','wb')
pkl.dump(ind_cacm.index,fichier_index)
fichier_index.close

fichier_index_inv=open('index_inverse_cacm.txt','wb')
pkl.dump(ind_cacm.index_inverse,fichier_index_inv)
fichier_index_inv.close()

fichier_index=open('index_cisi.txt','wb')
pkl.dump(ind_cisi.index,fichier_index)
fichier_index.close

fichier_index_inv=open('index_inverse_cisi.txt','wb')
pkl.dump(ind_cisi.index_inverse,fichier_index_inv)
fichier_index_inv.close()

# Evaluation des modèles sur CACM

## TestSet validation (60-40)

#### Séparation de l'ensemble des requetes en TrainingSet (60%) et TestSet (40%)

In [138]:
liste_q=list(parsedQuery_cacm.keys())
random.shuffle(liste_q)
ens_q=list(filter(lambda x: len(x.list_id_doc)>0 ,[parsedQuery_cacm[id_q] for id_q in liste_q]))
q_app=ens_q[:int(0.6*len(ens_q))]
q_test=ens_q[int(0.6*len(ens_q)):]

#### Instanciation des modèles

In [139]:
Vect5wCosine=Vectoriel(ind_cacm,Weighter5(ind_cacm),True)
Vect5wDotProd=Vectoriel(ind_cacm,Weighter5(ind_cacm),False)
okapi=Okapi(ind_cacm)
ml=ModeleLangue(ind_cacm)

### Apprentissage

#### Apprentissage du modèle OkapiBM25

In [140]:
GridForK1=np.arange(0.5,1.5,0.1)
GridForB=np.arange(0.5,1.5,0.1)
okapi.train(q_app,Precision(10),GridForK1,GridForB)

{'bestTrainingScore': 0.213, 'k1': 1.4, 'b': 1.0}

#### Apprentissage du modèle de langue

In [141]:
GridForLambda=np.arange(0,1.1,0.1)
ml.train(q_app,Precision(10),GridForLambda)

{'bestTrainingScore ': 0.084, 'Lambda': 1.0}

### Tests

In [142]:
models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
evaluation =EvalIRModel(q_test,models)

#### Comparaison des modèles en terme de Precision@10

In [145]:
evaluation.afficher(evaluation.ComparePrecision(10),'Precision@10')

Moyenne de  Precision@10  pour le modele  Vect5wCosine  =  0.195
Ecart type =  0.176
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  Vect5wDotProd  =  0.048
Ecart type =  0.105
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  OkapiBM25  =  0.214
Ecart type =  0.188
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  ModèleLangue  =  0.024
Ecart type =  0.087
------------------------------------------------------------


#### Comparaison des modèles en terme de Rappel@10

In [146]:
evaluation.afficher(evaluation.CompareRappel(10),'Rappel@10')

Moyenne de  Rappel@10  pour le modele  Vect5wCosine  =  0.123
Ecart type =  0.165
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  Vect5wDotProd  =  0.02
Ecart type =  0.048
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  OkapiBM25  =  0.132
Ecart type =  0.137
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  ModèleLangue  =  0.012
Ecart type =  0.044
------------------------------------------------------------


#### Comparaison des modèles en terme de FMesure@10

In [147]:
evaluation.afficher(evaluation.CompareFmesure(10),'FMesure@10')

Moyenne de  FMesure@10  pour le modele  Vect5wCosine  =  0.133
Ecart type =  0.124
------------------------------------------------------------
Moyenne de  FMesure@10  pour le modele  Vect5wDotProd  =  0.027
Ecart type =  0.065
------------------------------------------------------------
Moyenne de  FMesure@10  pour le modele  OkapiBM25  =  0.145
Ecart type =  0.12
------------------------------------------------------------
Moyenne de  FMesure@10  pour le modele  ModèleLangue  =  0.016
Ecart type =  0.058
------------------------------------------------------------


#### Comparaison des modèles en terme de AvgP

In [148]:
evaluation.afficher(evaluation.CompareAvgP(10),'AvgP')

Moyenne de  AvgP  pour le modele  Vect5wCosine  =  0.041
Ecart type =  0.102
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  Vect5wDotProd  =  0.004
Ecart type =  0.013
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  OkapiBM25  =  0.036
Ecart type =  0.048
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  ModèleLangue  =  0.002
Ecart type =  0.01
------------------------------------------------------------


#### Comparaison des modèles en terme de RR

In [149]:
evaluation.afficher(evaluation.CompareRR(),'ReciprocalRank')

Moyenne de  ReciprocalRank  pour le modele  Vect5wCosine  =  0.485
Ecart type =  0.398
------------------------------------------------------------
Moyenne de  ReciprocalRank  pour le modele  Vect5wDotProd  =  0.168
Ecart type =  0.306
------------------------------------------------------------
Moyenne de  ReciprocalRank  pour le modele  OkapiBM25  =  0.592
Ecart type =  0.439
------------------------------------------------------------
Moyenne de  ReciprocalRank  pour le modele  ModèleLangue  =  0.102
Ecart type =  0.291
------------------------------------------------------------


#### Comparaison des modèles en terme de NDCG@10

In [150]:
evaluation.afficher(evaluation.CompareNDCG(10),'NDCG@10')

Moyenne de  NDCG@10  pour le modele  Vect5wCosine  =  0.238
Ecart type =  0.226
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  Vect5wDotProd  =  0.057
Ecart type =  0.12
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  OkapiBM25  =  0.25
Ecart type =  0.215
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  ModèleLangue  =  0.024
Ecart type =  0.08
------------------------------------------------------------


##  Cross Validation ( 5 folds )

#### Séparation de l'ensemble des requtes en 5 folds

In [130]:
liste_q=list(parsedQuery_cacm.keys())
ens_q=np.array(list(filter(lambda x: len(x.list_id_doc)>0 ,[parsedQuery_cacm[id_q] for id_q in liste_q])))
np.random.shuffle(ens_q)
ens_splité=np.array_split(ens_q,5)
folds=list()
for i in range(len(ens_splité)):
    folds.append((np.concatenate([ens_splité[j] if i!=j else [] for j in range(len(ens_splité))],axis=None),ens_splité[i]))

#### Instanciation des modèles

In [131]:
Vect5wCosine=Vectoriel(ind_cacm,Weighter5(ind_cacm),True)
Vect5wDotProd=Vectoriel(ind_cacm,Weighter5(ind_cacm),False)
okapi=Okapi(ind_cacm)
ml=ModeleLangue(ind_cacm)

### Apprentissage

#### Apprentissage d'un modèle OkapiBM25 pour chaque fold

In [132]:
GridForK1=np.arange(0.5,1.5,0.1)
GridForB=np.arange(0.5,1.5,0.1)
param_okapi=list()
for i in range(len(folds)):
    param_okapi.append(okapi.train(folds[i][0],Precision(10),GridForK1,GridForB))

In [135]:
param_okapi

[{'bestTrainingScore': 0.21, 'k1': 1.4, 'b': 1.1},
 {'bestTrainingScore': 0.215, 'k1': 1.3, 'b': 1.0},
 {'bestTrainingScore': 0.214, 'k1': 1.4, 'b': 1.0},
 {'bestTrainingScore': 0.214, 'k1': 1.3, 'b': 0.9},
 {'bestTrainingScore': 0.219, 'k1': 1.4, 'b': 1.0}]

#### Apprentissage d'un modèle de langue pour chaque fold

In [133]:
GridForLambda=np.arange(0,1.1,0.1)
param_ml=list()
for i in range(len(folds)):
    param_ml.append(ml.train(folds[i][0],Precision(10),GridForLambda))

In [136]:
param_ml

[{'bestTrainingScore ': 0.066, 'Lambda': 1.0},
 {'bestTrainingScore ': 0.049, 'Lambda': 1.0},
 {'bestTrainingScore ': 0.067, 'Lambda': 1.0},
 {'bestTrainingScore ': 0.052, 'Lambda': 1.0},
 {'bestTrainingScore ': 0.064, 'Lambda': 1.0}]

## Tests 

### Pour chaque modele, on calcule la moyenne des scores sur les 5 folds de test

#### Comparaison des modèles en terme de Precision@10

In [156]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    precision=evaluation.ComparePrecision(10)
    dico_Precision={nom:([],[]) for nom in models.keys()}
    for key,value in precision.items():
        dico_Precision[key][0].append(value[0])
        dico_Precision[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_Precision.items()},'Precision@10')

Moyenne de  Precision@10  pour le modele  Vect5wCosine  =  0.2
Ecart type =  0.219
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  Vect5wDotProd  =  0.04
Ecart type =  0.066
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  OkapiBM25  =  0.19
Ecart type =  0.158
------------------------------------------------------------
Moyenne de  Precision@10  pour le modele  ModèleLangue  =  0.04
Ecart type =  0.066
------------------------------------------------------------


#### Comparaison des modèles en terme de Rappel@10

In [157]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    rappel=evaluation.CompareRappel(10)
    dico_Rappel={nom:([],[]) for nom in models.keys()}
    for key,value in rappel.items():
        dico_Rappel[key][0].append(value[0])
        dico_Rappel[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_Rappel.items()},'Rappel@10')

Moyenne de  Rappel@10  pour le modele  Vect5wCosine  =  0.196
Ecart type =  0.245
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  Vect5wDotProd  =  0.103
Ecart type =  0.212
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  OkapiBM25  =  0.176
Ecart type =  0.195
------------------------------------------------------------
Moyenne de  Rappel@10  pour le modele  ModèleLangue  =  0.103
Ecart type =  0.212
------------------------------------------------------------


#### Comparaison des modèles en terme de FMesure@10

In [159]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    fmesure=evaluation.CompareFmesure(10)
    dico_Fmesure={nom:([],[]) for nom in models.keys()}
    for key,value in fmesure.items():
        dico_Fmesure[key][0].append(value[0])
        dico_Fmesure[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_Fmesure.items()},'Fmesure@10')

Moyenne de  Fmesure@10  pour le modele  Vect5wCosine  =  0.146
Ecart type =  0.129
------------------------------------------------------------
Moyenne de  Fmesure@10  pour le modele  Vect5wDotProd  =  0.051
Ecart type =  0.097
------------------------------------------------------------
Moyenne de  Fmesure@10  pour le modele  OkapiBM25  =  0.141
Ecart type =  0.105
------------------------------------------------------------
Moyenne de  Fmesure@10  pour le modele  ModèleLangue  =  0.051
Ecart type =  0.097
------------------------------------------------------------


#### Comparaison des modèles en terme de AvgP

In [160]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    avgp=evaluation.CompareAvgP(10)
    dico_avgp={nom:([],[]) for nom in models.keys()}
    for key,value in avgp.items():
        dico_avgp[key][0].append(value[0])
        dico_avgp[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_avgp.items()},'AvgP')

Moyenne de  AvgP  pour le modele  Vect5wCosine  =  0.04
Ecart type =  0.056
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  Vect5wDotProd  =  0.033
Ecart type =  0.1
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  OkapiBM25  =  0.054
Ecart type =  0.098
------------------------------------------------------------
Moyenne de  AvgP  pour le modele  ModèleLangue  =  0.004
Ecart type =  0.011
------------------------------------------------------------


#### Comparaison des modèles en terme de ReciprocalRank

In [162]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    rr=evaluation.CompareRR()
    dico_RR={nom:([],[]) for nom in models.keys()}
    for key,value in rr.items():
        dico_RR[key][0].append(value[0])
        dico_RR[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_RR.items()},'Reciprocal Rank')

Moyenne de  Reciprocal Rank  pour le modele  Vect5wCosine  =  0.418
Ecart type =  0.406
------------------------------------------------------------
Moyenne de  Reciprocal Rank  pour le modele  Vect5wDotProd  =  0.171
Ecart type =  0.312
------------------------------------------------------------
Moyenne de  Reciprocal Rank  pour le modele  OkapiBM25  =  0.488
Ecart type =  0.38
------------------------------------------------------------
Moyenne de  Reciprocal Rank  pour le modele  ModèleLangue  =  0.177
Ecart type =  0.313
------------------------------------------------------------


#### Comparaison des modèles en terme de NDCG@10

In [163]:
for i in range(len(folds)):
    okapi=Okapi(ind_cacm,param_okapi[i]['k1'],param_okapi[i]['b'])
    ml=ModeleLangue(ind_cacm,param_ml[i]['Lambda'])
    models={'Vect5wCosine': Vect5wCosine,'Vect5wDotProd': Vect5wDotProd,'OkapiBM25': okapi,'ModèleLangue': ml}
    evaluation =EvalIRModel(folds[i][1],models)
    ndcg=evaluation.CompareNDCG(10)
    dico_NDCG={nom:([],[]) for nom in models.keys()}
    for key,value in ndcg.items():
        dico_NDCG[key][0].append(value[0])
        dico_NDCG[key][1].append(value[1])
evaluation.afficher({nom:(np.mean(stats[0]),np.mean(stats[1])) for nom, stats in dico_NDCG.items()},'NDCG@10')

Moyenne de  NDCG@10  pour le modele  Vect5wCosine  =  0.28
Ecart type =  0.289
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  Vect5wDotProd  =  0.122
Ecart type =  0.239
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  OkapiBM25  =  0.306
Ecart type =  0.25
------------------------------------------------------------
Moyenne de  NDCG@10  pour le modele  ModèleLangue  =  0.095
Ecart type =  0.169
------------------------------------------------------------
